<div style="text-align: center"><h1 style="text-decoration: underline;">DSML Project</h1></div>



This is the official Notebook of the DSML Project from Marc Rennefort, Kilian Lipinsky, Timo Hagelberg, Jan Behrendt-Emden and Paul Severin. In order to create this Project we used the following dataset: https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips-2023-2024-/n26f-ihde/about_data
<h4>1. Description</h4>
The goal of this Project is to predict the tip for ride-hailing in Chicago based on the travelled time and distance as well as the money spent, the weather and the information if the customer matched to a shared trip or not. 


In [40]:
#Note all your imports here
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime


In [42]:
data = pd.read_csv('Data/Chicago_RideHailing_Data.csv') #habt geduld, dauert beim laden

C:\Users\mrenn\AppData\Local\Temp\ipykernel_14492\1788640310.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/Chicago_RideHailing_Data.csv') #habt geduld, dauert beim laden


In [43]:
#some basic information for a better understanding of the data
data.head()
data.info()
data.isnull().sum()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27423899 entries, 0 to 27423898
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Trip Start Timestamp        object 
 2   Trip End Timestamp          object 
 3   Trip Seconds                float64
 4   Trip Miles                  float64
 5   Percent Time Chicago        float64
 6   Percent Distance Chicago    float64
 7   Pickup Census Tract         float64
 8   Dropoff Census Tract        float64
 9   Pickup Community Area       float64
 10  Dropoff Community Area      float64
 11  Fare                        float64
 12  Tip                         float64
 13  Additional Charges          float64
 14  Trip Total                  float64
 15  Shared Trip Authorized      object 
 16  Shared Trip Match           object 
 17  Trips Pooled                float64
 18  Pickup Centroid Latitude    float64
 19  Pickup Centroid Lon

Trip ID                              0
Trip Start Timestamp                 0
Trip End Timestamp                   0
Trip Seconds                      1638
Trip Miles                           1
Percent Time Chicago              6595
Percent Distance Chicago          7583
Pickup Census Tract           10481937
Dropoff Census Tract          10528302
Pickup Community Area          2404874
Dropoff Community Area         2488924
Fare                             68110
Tip                              68110
Additional Charges               68110
Trip Total                       68110
Shared Trip Authorized               1
Shared Trip Match                    1
Trips Pooled                         1
Pickup Centroid Latitude       2322890
Pickup Centroid Longitude      2322890
Pickup Centroid Location       2322890
Dropoff Centroid Latitude      2416867
Dropoff Centroid Longitude     2416867
Dropoff Centroid Location      2416867
dtype: int64

In [46]:
#Entfernen der Spalten, die wir nicht brauchen

data_cleaned = data.drop(columns = ['Percent Time Chicago', 'Percent Distance Chicago', 'Pickup Census Tract', 'Dropoff Census Tract',
                            'Pickup Community Area', 'Dropoff Community Area', 'Additional Charges', 'Trips Pooled', 
                            'Pickup Centroid Latitude', 'Pickup Centroid Location', 'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude', 'Dropoff Centroid Location', 'Pickup Centroid Longitude'])
data_cleaned.head()


,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tip,Trip Total,Shared Trip Authorized,Shared Trip Match
0,8255c0b48b877bda14fbc2f490b6e42f359c44ec,09/19/2023 06:15:00 PM,09/19/2023 07:15:00 PM,3109.0,23.5,32.5,0.0,40.70,False,False
1,8255c1529cdd828c3ce4e6bdc494c734e8432025,09/28/2023 12:15:00 PM,09/28/2023 12:45:00 PM,1640.0,14.7,30.0,0.0,38.74,False,False
2,8255c247359d6e148930ba860afa3282848c67ed,09/03/2023 05:15:00 PM,09/03/2023 05:30:00 PM,772.0,1.9,20.0,0.0,22.39,False,False
3,8255c37a57997b77f75da6af598a38fc28b4e832,09/13/2023 03:30:00 PM,09/13/2023 03:30:00 PM,452.0,1.1,10.0,0.0,14.07,False,False
4,8255c6b273a32b3a76d7dad03328abb229de7fbd,09/16/2023 01:00:00 PM,09/16/2023 01:00:00 PM,819.0,3.9,10.0,0.0,12.89,False,False


In [48]:
#drop all rows with null values
data_cleaned = data_cleaned.dropna(axis = 0)

In [56]:
#changing format of the date (Trip Start Timestamp)
data_cleaned['Trip Start Timestamp'] = pd.to_datetime(data_cleaned['Trip Start Timestamp'],  format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_cleaned['Trip End Timestamp'] = pd.to_datetime(data_cleaned['Trip End Timestamp'],  format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [58]:
data_cleaned.isnull().sum()
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27354156 entries, 0 to 27423897
Data columns (total 10 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Trip ID                 object        
 1   Trip Start Timestamp    datetime64[ns]
 2   Trip End Timestamp      datetime64[ns]
 3   Trip Seconds            float64       
 4   Trip Miles              float64       
 5   Fare                    float64       
 6   Tip                     float64       
 7   Trip Total              float64       
 8   Shared Trip Authorized  object        
 9   Shared Trip Match       object        
dtypes: datetime64[ns](2), float64(5), object(3)
memory usage: 2.2+ GB
